# Notebook for the creation of the grid lsh hashes for both datasets

Sheet that converts the extracted data from the data/chosen_data folder to hashes that will be stored in data/hashed_data/grid

In [12]:
# Importing nescessary modules
import os
import shutil

from schemes.grid_lsh import GridLSH
from utils import metafile_handler as mfh

from schemes.experiments import hashing

from multiprocessing import Pool


In [13]:
# Declaring global variables:

SHOULD_DELETE_OLD_FILES= True

OUTPUT_FOLDER_PORTO = "../data/hashed_data/grid/porto/"
OUTPUT_FOLDER_ROME = "../data/hashed_data/grid/rome/"

PORTO_DATA = "../data/chosen_data/porto/"
ROME_DATA = "../data/chosen_data/rome/"

P_MAX_LON = -8.57
P_MIN_LON = -8.66
P_MAX_LAT = 41.19
P_MIN_LAT = 41.14

R_MAX_LON = 12.53
R_MIN_LON = 12.44
R_MAX_LAT = 41.93
R_MIN_LAT = 41.88

---
## Porto LSH

Beginning with the porto set

In [14]:
# Run this cell to clear the chosen files in the PORTO folder

if SHOULD_DELETE_OLD_FILES:
    for filename in os.listdir(OUTPUT_FOLDER_PORTO):
        file_path = os.path.join(OUTPUT_FOLDER_PORTO, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print("Failed to remove %s. Reason: %s" % (file_path, e))

In [15]:
# Create Grid LSH objec for Porto

resolution = 0.25 # km
layers = 4
meta_file = "../data/chosen_data/porto/META-1000.TXT"

GridPorto = GridLSH("Porto G1", P_MIN_LAT, P_MAX_LAT, P_MIN_LON, P_MAX_LON, resolution, layers, meta_file, PORTO_DATA )



In [16]:
# Creating the hashes and saving them to output folder

hashes = GridPorto.compute_dataset_hashes()

# Saving the hashes to files
for key in hashes:
    with open(f'{OUTPUT_FOLDER_PORTO}/{key}.txt','w') as file:
        for hash in hashes[key]:
            file.write("%s\n" % hash)
        file.close()

# Copying the meta_files
meta_files = mfh.get_meta_files(PORTO_DATA)

for filename in meta_files:
    shutil.copy(PORTO_DATA + filename, OUTPUT_FOLDER_PORTO)

---
## Rome LSH

Continuing with the rome set

In [17]:
# Run this cell to clear the chosen files in the ROME folder

if SHOULD_DELETE_OLD_FILES:
    for filename in os.listdir(OUTPUT_FOLDER_ROME):
        file_path = os.path.join(OUTPUT_FOLDER_ROME, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print("Failed to remove %s. Reason: %s" % (file_path, e))

In [18]:
# Create Grid hash object for Rome and saves them to output folder. Also copies the metafiles denoting the different datasets

resolution = 0.25 # km
layers = 4
meta_file = "../data/chosen_data/rome/META-1000.TXT"

GridRome = GridLSH("Rome G1", R_MIN_LAT, R_MAX_LAT, R_MIN_LON, R_MAX_LON, resolution, layers, meta_file, ROME_DATA)


In [19]:
# Generate the hashes and save them to output folder

hashes = GridRome.compute_dataset_hashes()

# Saving the hashes to files
for key in hashes:
    with open(f"{OUTPUT_FOLDER_ROME}/{key}.txt", "w") as file:
        for hash in hashes[key]:
            file.write("%s\n" % hash)
        file.close()

# Copying the meta_files:
meta_files = mfh.get_meta_files(ROME_DATA)

for filename in meta_files:
    shutil.copy(ROME_DATA + filename, OUTPUT_FOLDER_ROME)

# Measuring run-times of hash generation
The cells below are created to measure the time-efficiency of the hash computation


In [20]:
# Measuring Grid compuation over Porto
with Pool() as pool:
    result = pool.map(hashing.fun_wrapper_p_grid, [[1000, 0.25, 4] for _ in range(10)])
print(result)
print(f"Ran the hash computation {len(result)} times. Fastest runtime: {min(result)}s")

[[0.1875], [0.140625], [0.1875], [0.25], [0.171875], [0.1875], [0.171875], [0.171875], [0.203125], [0.203125]]
Ran the hash computation 10 times. Fastest runtime: [0.140625]s


In [21]:
# Measuring Grid compuation over Rome
with Pool() as pool:
    result = pool.map(hashing.fun_wrapper_r_grid, [[1000, 0.25, 4] for _ in range(10)])
print(result)
print(f"Ran the hash computation {len(result)} times. Fastest runtime: {min(result)}s")

[[0.40625], [0.453125], [0.421875], [0.328125], [0.53125], [0.46875], [0.46875], [0.453125], [0.25], [0.390625]]
Ran the hash computation 10 times. Fastest runtime: [0.25]s
